In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd
import numpy as np

In [ ]:
!pwd

/content


In [3]:
# about rating
path_rating = 'drive/MyDrive/Notebooks/ml-100k/u.data'
# about movie
path_movie = 'drive/MyDrive/Notebooks/ml-100k/u.item'
# about user
path_user = 'drive/MyDrive/Notebooks/ml-100k/u.user'

In [5]:
def parse(s):
  seq = s.split(' | ')
  return list(map(lambda s: s.replace(' ', '_'), seq))

parse('user id | age | gender | occupation | zip code')

['user_id', 'age', 'gender', 'occupation', 'zip_code']

In [6]:
user_col = parse('user id | age | gender | occupation | zip code')
users = pd.read_csv(path_user, sep='|', names=user_col)
users = users.set_index(user_col[0])
users.head(5)

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [7]:
# movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western

movie_col = parse("movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western")
movies = pd.read_csv(path_movie, sep='|', names=movie_col, encoding='ISO-8859-1')
movies = movies.set_index(movie_col[0])
movies.head(5)

,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [9]:
# user id | item id | rating | timestamp
rating_col = parse('user id | item id | rating | timestamp')

ratings = pd.read_csv(path_rating, sep='\t', names=rating_col, encoding='ISO-8859-1')
ratings = ratings.set_index(rating_col[0])
ratings.head(5)

,item_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [10]:
ratings_mean = ratings.groupby(rating_col[1]).mean()
dir(ratings_mean)
ratings_mean.sort_values(by=rating_col[2], ascending=False)

,rating,timestamp
item_id,,
814,5.0,886302261.0
1599,5.0,880142614.0
1201,5.0,891383687.0
1122,5.0,883326498.0
1653,5.0,889489913.0
...,...,...
1568,1.0,885547222.0
1567,1.0,885547123.0
1566,1.0,885546248.0


In [12]:
movie_ratings_mean = ratings_mean.groupby(["item_id"])["rating"].mean()
movie_ratings_mean

item_id
1       3.878319
2       3.206107
3       3.033333
4       3.550239
5       3.302326
          ...   
1678    1.000000
1679    3.000000
1680    2.000000
1681    3.000000
1682    3.000000
Name: rating, Length: 1682, dtype: float64

In [14]:
movie_rating_sort = movie_ratings_mean.sort_values(ascending=False)[:5]
movie_rating_sort

item_id
1293    5.0
1467    5.0
1653    5.0
814     5.0
1122    5.0
Name: rating, dtype: float64

In [17]:
recom_movies = movies.loc[movie_rating_sort.index]
recom_movies['movie_title']

item_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: movie_title, dtype: object

오차 계산  
**RMSE (Root Mean Square Error)** 사용

In [22]:
def RMSE(answer, prediction):
  e = (np.array(answer) - np.array(prediction)) ** 2
  return np.sqrt(np.mean((e)))

In [27]:
rmse = []
for user in set(ratings.index):
  answer = ratings.loc[user]['rating']
  prediction = movie_ratings_mean.loc[ratings.loc[user]['item_id']]
  accuracy = RMSE(answer, prediction)
  rmse.append(accuracy)

print(np.mean(rmse))

0.996007224010567


item_id 당 받은 평가의 갯수가 상위 1000위 이하인 영화들 중에서 추천 (전체 : 1682개)  
표본이 적을 경우 신뢰도가 떨어지므로

In [30]:
ratings_1000 = ratings.groupby(['item_id']).size().sort_values(ascending=False)[:1000]
ratings_1000

item_id
50      583
258     509
100     508
181     507
294     485
       ... 
1419     16
854      16
453      16
997      16
256      16
Length: 1000, dtype: int64

In [32]:
filtered_ratings = ratings[ratings['item_id'].isin(ratings_1000.index)]
filtered_ratings

,item_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
244,51,2,880606923
166,346,1,886397596
298,474,4,884182806
...,...,...,...
880,476,3,880175444
716,204,5,879795543
276,1090,1,874795795


In [36]:
filtered_ratings_mean = filtered_ratings.groupby(['item_id'])['rating'].mean()
filtered_ratings_mean

item_id
1       3.878319
2       3.206107
3       3.033333
4       3.550239
5       3.302326
          ...   
1407    2.250000
1411    2.571429
1419    2.125000
1444    2.473684
1478    2.722222
Name: rating, Length: 1000, dtype: float64

In [37]:
filtered_rating_mean_sort = filtered_ratings_mean.sort_values(ascending=False)[:10]
filtered_rating_mean_sort

item_id
408    4.491071
318    4.466443
169    4.466102
483    4.456790
114    4.447761
64     4.445230
603    4.387560
12     4.385768
50     4.358491
178    4.344000
Name: rating, dtype: float64

In [38]:
movies.loc[filtered_rating_mean_sort.index]

,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item_id,,,,,,,,,,,,,,,,,,,,,,,
408,"Close Shave, A (1995)",28-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Close%20Shave...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
318,Schindler's List (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Schindler's%2...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
169,"Wrong Trousers, The (1993)",01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Wrong%20Trous...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
483,Casablanca (1942),01-Jan-1942,NaN,http://us.imdb.com/M/title-exact?Casablanca%20...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
114,Wallace & Gromit: The Best of Aardman Animatio...,05-Apr-1996,NaN,http://us.imdb.com/Title?Wallace+%26+Gromit%3A...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64,"Shawshank Redemption, The (1994)",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Shawshank%20R...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
603,Rear Window (1954),01-Jan-1954,NaN,http://us.imdb.com/M/title-exact?Rear%20Window...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
12,"Usual Suspects, The (1995)",14-Aug-1995,NaN,http://us.imdb.com/M/title-exact?Usual%20Suspe...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


여태까지는 사용자를 고려하지 않음.  
앞으로는 사용자의 특성을 고려함.
